# Advanced DB Topics assignment
### DB initialization

Connect to database `image_db`. Drop `image` table if it exists and rebuild it.

In [1]:
import psycopg2
import numpy as np

conn = psycopg2.connect(database="image_db", 
                        user = "postgres", 
                        password = "admin", 
                        host = "localhost", 
                        port = "5432")

print("Opened database successfully")

cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS images;''')

print("Table \'images\' removed successfully")

cur.execute('''CREATE TABLE public.images
               (
                   id bigint NOT NULL,
                   filename text NOT NULL,
                   filepath text COLLATE pg_catalog."default" NOT NULL,
                   descriptor_vector bytea NOT NULL
               );''')

print ('Table \'images\' created successfully')

conn.commit()
conn.close()

Opened database successfully
Table 'images' removed successfully
Table 'images' created successfully


### Feature extraction

Column preperation and feature extraction

Parsing files for images and construct the `id`, `filepath` and `groundtruth` columns

In [2]:
import glob
import os 
cur_dir = ''#os.path.dirname(os.path.realpath(__file__))
img_dir = os.path.join(cur_dir, 'static', 'images', 'celebrities')

img_list = [f for p in os.walk(img_dir) for f in glob.glob(os.path.join(p[0], '*.jpg')) if p!='UPLOADS']
img_id = [f.split(os.sep)[-1] for f in img_list]

In [3]:
from  FeatureExtractor import FeatureExtractor
from tqdm import tqdm

descriptor_vectors = []
for im in tqdm(img_list):
#    print(im)
   img_features = FeatureExtractor(im, use_face_mask=False)
   descriptor_vector = img_features.feature_extractor()
   descriptor_vectors.append(descriptor_vector)
print('Finished image feature extraction')

descriptor_vectors = np.asarray(descriptor_vectors)
descriptor_vector.shape

100%|██████████| 1588/1588 [01:22<00:00, 19.30it/s]

Finished image feature extraction


(2048,)

Populate `images` table

In [4]:
import psycopg2

conn = psycopg2.connect(database="image_db", 
                        user = "postgres", 
                        password = "admin", 
                        host = "localhost", 
                        port = "5432")

print("Opened database successfully")

cur = conn.cursor()

print("Write data in db")
for i in tqdm(range(len(img_list))):
   cur.execute("INSERT INTO IMAGES (ID, FILENAME, FILEPATH, DESCRIPTOR_VECTOR) \
         VALUES (%s,%s,%s,%s )", (i,
                                     img_id[i], 
                                     img_list[i], 
                                     descriptor_vectors[i].tostring()))

conn.commit()
print ("Records created successfully")
conn.close()

  9%|▉         | 141/1588 [00:00<00:01, 1405.98it/s]

Opened database successfully
Write data in db


100%|██████████| 1588/1588 [00:01<00:00, 1513.67it/s]

Records created successfully


Retrieve all table records

In [5]:
from datetime import datetime
import psycopg2
import numpy as np

conn = psycopg2.connect(database="image_db", 
                        user = "postgres", 
                        password = "admin", 
                        host = "localhost", 
                        port = "5432")

print("Opened database successfully")
#Creating a cursor object using the cursor() method
cur = conn.cursor()
# Setup query
sql = '''SELECT * from images'''
start = datetime.now()
print('Start retrieving query')
#Executing the query
cur.execute(sql)
#Fetching 1st row from the table
request = cur.fetchall();
print('DB transaction finished in', datetime.now() - start)

#Closing the connection
conn.close()

Opened database successfully
Start retrieving query
DB transaction finished in 0:00:00.215048
